<a href="https://colab.research.google.com/github/simonEllershaw/latent_variable_OOD/blob/main/transformer_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
print('hello world')

hello world
